In [ ]:
### IMPORTS ###
import torch
import datasets

import os
import spacy
import numpy as np
import pandas as pd

from accelerate import Accelerator
from huggingface_hub import get_full_repo_name, Repository, notebook_login
from tqdm.auto import tqdm
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification, pipeline, Trainer, TrainingArguments

from pathlib import Path

# exploration

So original is just the post with all annotations, and SCT and MEDDRA are different types of tags, with SCT non drug terms and Meddra drug terms

In [ ]:
# explore data

# original example
pd.read_csv("./data/CADEC.v2/cadec/original/ARTHROTEC.1.ann", sep='\t', header=None)

In [ ]:
#  MedDRA example
pd.read_csv("./data/CADEC.v2/cadec/meddra/ARTHROTEC.1.ann", sep='\t', header=None)

In [ ]:
# SCT wxamples
pd.read_csv("./data/CADEC.v2/cadec/sct/ARTHROTEC.1.ann", sep='\t', header=None)

In [ ]:
annotation_directory = Path('./data/CADEC.v2/cadec/original')
unique_drugs = set()

for file_path in annotation_directory.glob('*.ann'):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            if '\tDrug ' in line:
                drug_name = line.split('\t')[2].strip()
                unique_drugs.add(drug_name)

print('unique drug names: \n')
for i in unique_drugs:
    print(i)

print ("\n num: ", len(unique_drugs))

In [ ]:
#drugs from paper

drugs = ['ARTHROTEC', 'CAMBIA', 'CATAFLAM', 'DICLOFENAC-POTASSIUM', 'DICLOFENAC-SODIUM',
         'FLECTOR', 'LIPITOR', 'PENNSAID', 'SOLARAZE', 'VOLTAREN', 'VOLTAREN-XR', 'ZIPSOR']

In [ ]:
reports_per_drug = dict()


print('amount of reports for each drug: \n')

# iterate over each drug
for drug in drugs:
    reports_directory = Path('./data/CADEC.v2/cadec/original')
    
    drug_files = reports_directory.glob(drug + '*.ann')
    num_reports_for_current_drug = sum(1 for file in drug_files)
    reports_per_drug[drug] = num_reports_for_current_drug
    print(f'{drug} --> {num_reports_for_current_drug}')